# Talk to construction PDF page with `gpt-4o`

### Instructions

- Azure OpenAI env variables
- brew install poppler

NOTE: uses same data as extract_pdf_data_llm

In [1]:
import os
from rich import print


def read_image_as_base64(image_path: str) -> str:
    import base64

    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    return image_data

In [2]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage


llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT"),
)

In [3]:
def talk_to_image(page_content_base64: str, question: str):
    prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": question},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{page_content_base64}"},
                },
            ],
        ),
    ]
)
    
    chain = prompt | llm

    return chain.invoke({})


![Page 4](./var/extract_pdf_data_llm/out4.jpg)

In [4]:
def talk_to_page_4(question: str):
    page_base64 = read_image_as_base64("./var/extract_pdf_data_llm/out4.jpg")

    print(talk_to_image(page_base64, question).content)

In [6]:
talk_to_page_4("How many bedrooms?")

The floor plan shows a total of 3 bedrooms, labeled as Bed 1, Bed 2, and Bed 3.

In [8]:
talk_to_page_4("What is the perimeter of the building?")

To calculate the perimeter of the building, we need to add up the lengths of all the outer walls. The given 
dimensions are in millimeters. Let's identify and sum up these dimensions:

1. Horizontal dimensions at the top:
   - 450 + 2,150 + 70 + 1,500 + 1,270 + 4,100 + 70 + 2,215 + 450 = 12,275 mm

2. Right vertical dimensions:
   - 450 + 7,300 + 450 = 8,200 mm

3. Bottom horizontal dimensions:
   - 450 + 3,200 + 70 + 1,100 + 70 + 4,785 + 70 + 2,700 + 70 + 4,220 + 450 = 17,685 mm

4. Left vertical dimensions:
   - 70 + 400 + 3,420 + 400 + 70 + 2,320 + 70 = 6,750 mm

The total perimeter is the sum of these dimensions:
- Top: 12,275 mm
- Right: 8,200 mm
- Bottom: 17,685 mm
- Left: 6,750 mm

Perimeter = 12,275 + 8,200 + 17,685 + 6,750 = 44,910 mm

To convert millimeters to meters:
44,910 mm / 1,000 = 44.91 meters

Therefore, the perimeter of the building is 44.91 meters.

In [14]:
talk_to_page_4("What are the dimensions of the image? What is the position (x, y in pixels) for Bedroom 1?")

The dimensions of the image are 1920 pixels in width and 1080 pixels in height.

The position (x, y in pixels) for Bedroom 1 (Bed 1) is approximately (350, 430).

In [16]:
talk_to_page_4("Extract all data from image, and place into JSON")

```json
{
  "notes": [
    "Lift off hinges to wc door",
    "Provide grab rails where shown",
    "Contact floor waste position to be confirmed",
    "Mechanical vents to provide natural ventilation",
    "Reinforced concrete slab with AHD",
    "MP4 pressures to timber & 3000mm inset slab",
    "All glazed panels to 3000mm above floor",
    "Provide a step down to outdoor area as per builders location",
    "Roughing of stud positions for linings indicated on basement",
    "Utility to be laid on fiber cement",
    "Hold coat of finish to fiber cement @ 1500mm (3000mm on higher level)",
    "Where ventilation is shown on the plan",
    "Ceiling connection to be reduced to 800mm height or as per greater to be restricted to 1.2m",
    "150mm opening sliding windows with keyed lever locks, awning shall have restrictors"
  ],
  "rooms": [
    {
      "name": "Bed 1",
      "type": "carpet",
      "dimensions": "2.870 x 3.540"
    },
    {
      "name": "Ens",
      "type": "unspecified"
    },
    {
      "name": "WIR",
      "type": "unspecified"
    },
    {
      "name": "Kitchen",
      "type": "unspecified"
    },
    {
      "name": "Dining",
      "type": "tiles",
      "dimensions": "1.5"
    },
    {
      "name": "Living",
      "type": "tiles",
      "dimensions": "5.2 x 4.0"
    },
    {
      "name": "Bed 2",
      "type": "carpet",
      "dimensions": "3.0 x 2.7"
    },
    {
      "name": "Bath",
      "type": "unspecified",
      "dimensions": "2.0 x 2.0"
    },
    {
      "name": "Family",
      "type": "tiles",
      "dimensions": "3.2 x 3.4"
    },
    {
      "name": "Bed 3",
      "type": "carpet",
      "dimensions": "3.2 x 2.7"
    },
    {
      "name": "Alfresco",
      "type": "timber decking",
      "dimensions": "3.5 x 4.0"
    }
  ],
  "contacts": {
    "phone": "(07) 5562 6177",
    "mobile": "0408 701 623",
    "website": "www.thehousedesigners.com.au",
    "address": "1/7 Strathaird Road Bundall QLD 4217",
    "postal_address": "PO Box 2327 Nerang DC QLD 4210",
    "qbcc": "124038"
  },
  "area_calculations": {
    "living_lower": "29.56 sqm",
    "living_upper": "119.98 sqm",
    "garage": "36.08 sqm",
    "porch": "3.98 sqm",
    "alfresco": "12.31 sqm",
    "total": "201.91 sqm"
  },
  "drawing_info": {
    "print_date": "April 2015",
    "drawing_no": "5 of 12",
    "design_name": "Outrigger 201",
    "scale": "1:100 (A3)",
    "drawn_by": "NB",
    "checked_by": "MB"
  },
  "plan_info": {
    "title": "Floor Plan - Upper",
    "project": "Outrigger 201",
    "drawing_type": "Working Drawings"
  }
}
```